In [23]:
import cv2
import numpy as np
import pytesseract
import pandas as pd
import re
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED) 

In [40]:
class Cleaner:
    def __init__(self, config):
        self.config = config
        pass
    def load(self, path):
        return cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    def preProcess(self, image):
        image = cv2.resize(image, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)
        #image = cv2.medianBlur(image,self.blurWindow)
        #thresholding
        #image = cv2.bilateralFilter(image,9,75,75)
        #image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        #kernel = np.ones((3,3),np.uint8)
        #image = cv2.dilate(image, kernel, iterations = 1)
        #image = cv2.erode(image, kernel, iterations = 1)
        #image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
        #image = cv2.Canny(image, 100, 200)
        return image
    def OCR(self, image):
        text = pytesseract.image_to_string(image, config=self.config)
        return text
    def extract(self, text):
        df = pd.DataFrame(columns=["arrêt", "date", "juridiction"])
        m = re.findall(r"LA COUR.+?([A-Za-z].+?)\n*?Du(.+?\d{3}.+?)\.?\s?(—|-)(.+?)\.?(—|,)", text.replace("\n", ""))
        for arret, date, _,region,_ in m:
            df = df.append({'arrêt' : arret , "date": date, "juridiction" : region},ignore_index=True)
        del m
        return df
    def clean(self, path):
        return self.OCR(self.preProcess(self.load(path)))
    def show(self, image):
        plt.imshow(image, cmap= "gray")
        plt.show()
        

In [41]:
c = Cleaner('--oem 3 --psm 1' )

In [8]:
text = c.clean("test5.jpg")

In [18]:
df = c.extract(text)

In [20]:
from spellchecker import SpellChecker
spell = SpellChecker(language='fr')
misspelled = spell.split_words(df["arrêt"][0])
misspelled = spell.unknown(misspelled)
for word in misspelled:
    print(word, spell.correction(word))

dedroit detroit
cn un
oii oui
rédhibitoire rédhibitoire
quela quel
vraison raison
cise mise
ott ont
maniérede maniére
vani van
devaitétre devaitétre
cingmyriamétres cingmyriamétres
etle elle
fart fait
matiére matière
oit dit
existeentre existeentre
rivée privée
précitée précise
partiesn parties
vespéce espéce
intentée inventée
ot et


In [2]:
import unicodedata

from bs4 import BeautifulSoup
import sqlite3
import requests
import re
import json

def get_pagination(ark):
    """fonction qui renvoie le nombre de pages d'un documents"""
    #print(cpt,"--> https://gallica.bnf.fr/"+i+"/f7/highres")
    req = requests.get("https://gallica.bnf.fr/services/Pagination?ark=" + ark)
    p = req.content
    pages = re.search(r"<nbVueImages>(\d*)</nbVueImages>", str(p))
    return pages

In [4]:
a = get_pagination("12148/bpt6k57460598")

In [32]:
f = open("test"+".jpg","wb")
f.write(r.content)

530083

In [42]:
text = c.clean("../data/test/test-7.jpg")

In [39]:
c.extract(text)

,arrêt,date,juridiction
0,Par les motifs exprimes dans Je jugement du ti...,"7, juillet 1829",C. r. Nimes.


In [43]:
print(text)

Lois et Decisions diverses.

nom (1). (Ordomn. de 1555; ordonn. de 1629, art. 2115 Joi du
31 germinal an 11.)
(De La Fare—C., Cabot-La Fare. )

Depuis environ un siécle Jes membres de la famille Cabot étaient
danst ‘usage d’ajouter 4 leur nom celui de La Fare, nom d'un domaine
noble quils avaient acquis ancicnnement — En 1827, le cardinal
et les marquis a Fare formérent contre les sieues Gabot une de-
mande tendante A ce qu'il leur fat fait défense de prendre 2 Pavenir
Je nom de La Fave. ur cette demande est intervenu au Wibunal
de Florac le jngement suivant, qui fait sullisamment connaitre Jes

 

 

de Li

 

 

  

moyens plaidés respectivement.

« Attendu qu'il n’est pas contesté que la famille des demandeurs
ct ses différentes branches n’aicht la propritté et la possession , de-
puis un temps immémorial, du nom de Lu Mare, et des tit
lifications honorifiques : qu'elle est, par suite, bien fondée & s’op-
tui dung autre

 

 

scl qua-

   

race que ce nom devienne exclusivemcnt ce

In [46]:
a.groups()[0]

'312'